### Creating the seed data set

Starting from complete trEMBL dataset <span style='background:#f7f3f7;padding:0.4em;border-radius:2px; border:solid bgrey 1px'>arwen:/mobi/group/NOX_CH/data/uniprot_trembl.fasta.gz</span> which is a symbolic link for `arwen:/mobi/group/databases/flat/uniprot_trembl_2019_02.fasta.gz`
 *  Split the dataset in small volumes
     * script: <span style="color:green">**split.py**</span>
     * Usage:
     Create and go to the `/mobi/group/NOX_GL/volumes` 
```console
    ROOT_DIR=/mobi/group/NOX_CH
    $ROOT_DIR/scripts/split.py $ROOT_DIR/data/uniprot_trembl.fasta.gz
```

 * Run the HMMR and TMHMM annotations
    * script: <span style="color:green">**runHMMR_slurm.sh**</span>
    * Usage:  
  
```console
    mkdir $ROOT_DIR/seedSet
    mkdir $ROOT_DIR/seedSet/work
    $ROOT_DIR/scripts/runHMMR_slurm.sh $ROOT_DIR/volumes $ROOT_DIR/seedSet/work $ROOT_DIR/data/profiles
```

 * Use this notebook to parse the _work_ folder (see **Parsing all data files** section)

    * Filter-out non eukaryotic entries and dump the corresponding fasta sequence in folder <span style='background:#f7f3f7;padding:0.4em;border-radius:2px; border:solid bgrey 1px'>/mobi/group/NOX_GL/seedSet/NOX_noEukaryota</span>


 * Preparing folders/sbatch scripts for pairwise N&W across the set of __NOX_noEukaryota__ fasta sequences
    * script: <span style="color:green">**runEMBOSS_slurm.sh**</span>
    * Usage:
```console
$ROOT_DIR/scripts/runEMBOSS_slurm.sh $ROOT_DIR/seedSet/NOX_noEukaryota NOX_noEukaryota $ROOT_DIR/seedSet/NOX_noEukaryota_needlePairwise_work
```

 * Concatenate all fasta sequences in a single file and perform a full PFAM annotation
```console
     cat $ROOT_DIR/seedSet/NOX_noEukaryota/*.fasta > NOX_noEukaryota.mfasta
     hmmscan NOX_noEukaryota.mfasta /mobi/group/databases/hmmr/Pfam-A.hmm > NOX_noEukaryota_hmmscan.out
```


 * Enrich the datacontainer with these new annotation, see **Read in additional PFAM annotations // Erase previous** section


 * Use the [Taxonomy notebook](http://localhost:8888/notebooks/NOX/Taxonomy.ipynb) to output a hierarchal tree
     * link the output json file as $latest.json$


 * Start adhoc http server
   
   Go to `~/work/projects/NOX`
```console
node index.js
``` 

* Visualize w/ D3 at `localhost:9615`
 
### Creating the extended data set


* Perform a psiblast on fasta files present in <span style='background:#f7f3f7;padding:0.4em;border-radius:2px; border:solid bgrey 1px'>arwen:/mobi/group/NOX_GL/seedSet/NOX_noEukaryota</span>

    * Create the `extendedSet` folder

    * script: <span style="color:green">**runPsiBlast_slurm.sh**</span>
    * Usage:
```console
$ROOT/scripts/runPsiBlast_slurm.sh $ROOT/seedSet/NOX_noEukaryota $ROOT/extendedSet/psiblastWork
```

* Browse all the psiblast workfolder and eliminate strictly identical proteins
    * Go to `$ROOT/extendedSet`
    * script:<span style="color:green">**makePsiBlastNR.py**</span>
    * Usage:
```console
$ROOT/scripts/makePsiBlastNR.py ./psiblastWork ./NOX_noEukaryota_PB_NR.fasta > makePsiBlastNR.log
```
* Perform a full PFAM annotation
```console
hmmscan NOX_noEukaryota_PB_NR.fasta /mobi/group/databases/hmmr/Pfam-A.hmm > NOX_noEukaryota_PB_NR_hmmscan.out
```

In [6]:
%matplotlib inline
import sys, os
sys.path.append("/Users/guillaumelaunay/work/DVL/python3/pyproteinsExt/src")
sys.path.append("/Users/guillaumelaunay/work/DVL/python3/pyproteins/src")
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import gzip, io
import urllib.request

def mFastaParseZip(inputFile):
    data = None
    with io.TextIOWrapper(gzip.open(inputFile, 'r')) as f:
        data = mFastaParseStream(f)
    return data

def mFastaParseUrl(url):
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()
    #mFastaParseStream(fp)
    mystr = mybytes.decode("utf8")
    fp.close()
    data = mFastaParseStream(mystr.split('\n'))
    
#    print(mystr)
    return data

def mFastaParseStream(stream):
    
    data = {}    
    headPtr = ''
    for line in stream:
        #print (line)
        if line == '':
            continue
        s = line.replace('\n','')
        if s.startswith('>'):
            headPtr = s.split()[0][1:]
            
            if headPtr in data:
                raise ValueError('Smtg wrong')
            data[headPtr] = {'header': s, 'sequence' : '' }
            
            continue
        data[headPtr]['sequence'] += s
    return data

#mFastaParseUrl('http://www.uniprot.org/uniprot/S4Z6V5.fasta')
#data = mFastaParse('/Volumes/arwen/home/ygestin/prositetask-backup/alignTrembl/bibl/Trembl_47/Trembl_47.fasta.gz')
#test=None
#with open('/Volumes/arwen/mobi/group/NOX_GL/work/uniprot_trembl_v11/hmmsearch.fasta', 'r') as f:
#    test = mFastaParseStream(f)

In [3]:
import re

def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)
    
    
reTMH = re.compile('^(\# ){0,1}([\S]+)[\s]+([\S].*)[\s]+([\d\.]+)$')
def loadTMHMM(lDir):
    
    fastaContainer = None
    with open( lDir+ '/hmmsearch.fasta', 'r') as f:
        fastaContainer = mFastaParseStream(f)
    
    file = lDir+ '/tmhmm.out'
    data = {}
    with open(file, 'r') as f:
        for l in f:
            m = reTMH.search(l)
            if m:
                _id = m.groups()[1] 
                if _id not in data:
                    if _id not in fastaContainer:
                        raise ValueError("Misisng fasta for tmhmm prediction")
                    data[_id] = {'hCount':0 ,
                                'helix':[], 'fasta' : fastaContainer[_id],
                                'mask': '-' * len(fastaContainer[_id]['sequence'])
                                }
                
                if not m.groups()[2].startswith('TMHMM2'):
                    data[_id][re.sub('[\s]*:[\s]*$', '',m.groups()[2])] = num(m.groups()[3])
                    continue
                
                
                m2 = m.groups()[2].split('\t')
                if not m2:
                    raise ValueError('could not parse helix line')
                helixCoor =  {'volume' : m2[1], 
                              'start'  : num(m2[2].replace(' ', '')),
                              'stop'   : num(m.groups()[3]) 
                            }
                data[_id]['helix'].append(helixCoor)
                
                
                data[_id]['helix'].append(helixCoor)
                #print (data[_id]['mask']) 
                l_1 = len(data[_id]['mask'])
                buf = list(data[_id]['mask'])
                symbol = None
                if helixCoor['volume'] == 'TMhelix':
                    data[_id]['hCount'] += 1
                    #symbol = 'H'
                    symbol = str(data[_id]['hCount']) if data[_id]['hCount'] < 10 else str(data[_id]['hCount'])[-1]
                elif helixCoor['volume'] == 'inside':
                    symbol = 'i'
                elif helixCoor['volume'] == 'outside':
                    symbol = 'e'
                else :
                    raise ValueError("unknown symbol " + helixCoor['volume'])

                i=helixCoor['start'] - 1
                j=helixCoor['stop']
                #print(i,j,len(buf))
                toAdd = symbol * (j - i)
                buf[i:j] =  list(toAdd)#helixCoor['stop'] - helixCoor['start'] + 1
                data[_id]['mask'] = ''.join(buf)
                if len(data[_id]['mask']) != l_1:
                    print("ERROR ", _id, l_1, len(data[_id]['mask']), '>>', i, j, '<<')
                    print (len(buf[i:j]), len(list(toAdd)), symbol, '-->', toAdd )
                #print(data[_id]['mask'])
    
    #        Hcluster(data)
    return data
#d = loadTMHMM('/Volumes/arwen/home/ygestin/prositetask-backup/alignTrembl/bibl/Trembl_47')
#d = loadTMHMM('/Volumes/arwen/mobi/group/NOX_GL/work_sample/uniprot_trembl_v11')
#d

In [4]:
def HIS_clust(data, min=2, max=7):
    for _id in data:
        data[_id]['Htest'] = {'status' : False, 'data' : [] }

        #Discard unwanted numbe of helices
        if data[_id]['hCount'] < min or data[_id]['hCount'] > max:
            print('Wrong helices number ', _id, data[_id]['hCount'])
            continue
        
        H_status = []
        iMax = len(data[_id]['mask'])
        # internal error check
        if len(data[_id]['mask']) != len(data[_id]['fasta']['sequence']) :
            print( len(data[_id]['mask']), len(data[_id]['fasta']['sequence']) )
            print(_id, data[_id])
            raise ValueError("")
        # Select only residues that are Histidine within TMH
        for i in range(0, iMax):
            if data[_id]['mask'][i] == "i" or  data[_id]['mask'][i] == "e":
                continue
            if not data[_id]['fasta']['sequence'][i] == "H":
                continue
            H_status.append( [i, data[_id]['mask'][i], False] )
        # Pairwise comparaison between Histidine of the same helix, marking pairs separated by 12 to 14 residues
        for i in range (0, len(H_status) - 1):
            for j in range (i + 1, len(H_status)):
                if H_status[i][1] != H_status[j][1]:
                    continue
                d = H_status[i][0] - H_status[j][0]
                if d >= 12 or d <= 14:
                    H_status[i][2] = True
                    H_status[j][2] = True
        
        #print(H_status)
        # Only keep marked histidine
        H_status = [ x for x in H_status if x[2] ]
        # Create a dicitinary where keys are Helices numbers
        H_groups = {}
        for x in H_status:
            if not x[2]:
                continue
            if x[1] not in H_groups:
                H_groups[x[1]]=[]
            H_groups[x[1]].append(x)
        
        # The test is passed if at least two distinct helices feature at least one correctly spaced histidine pair
        # ie : if the helice dictionary has more than 1 entrie
        #print(H_status)
        #print("-->", H_groups)
        HisTestBool = True if len(H_groups) > 1 else False
        
        data[_id]['Htest']['status'] = HisTestBool
        data[_id]['Htest']['data'] = H_groups
    return data

#m = HIS_clust(d)
#print(len([ m[x] for x in m if m[x]['Htest']['status'] ]), len(m))

# Parsing all data files 

### Parsing HMMR data
NB: There are stdout of 3 consecutive hmmr calls

All in a single **data** container

In [7]:
import pyproteinsExt.hmmrContainerFactory as hm
import glob
dataDir=glob.glob('/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v*')

data = hm.parse(inputFile=dataDir[0] + '/hmmsearch.out')
i=0
for iDir in dataDir[1:]:
    print(iDir)
    data += hm.parse(inputFile=iDir + '/hmmsearch.out')
    i += 1
    #if i == 10:
    #    break

/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v115
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v7
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v50
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v27
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v186
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v168
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v216
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v86
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v68
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v127
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v150
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v15
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v62
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v224
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v195
/Volumes/arwen/mob

/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v119
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v180
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v56
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v21
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v1
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v164
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v113
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v93
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v132
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v145
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v138
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v99
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v77
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v209
/Volumes/arwen/mobi/group/NOX_GL/seedSet/work/uniprot_trembl_v203
/Volumes/arwen/mo

## Loading TMHMM data

In [ ]:
dataTMHMM = {}
for lDir in dataDir:
    d = loadTMHMM(lDir)
    if set( dataTMHMM.keys() ) & set( d.keys() ):
        print('doublons')
    dataTMHMM.update(d)

dataTMHMM = HIS_clust(dataTMHMM)

##### Transform a PFAM domain indexed data structure in a protein indexed data structure
Then filter out the protein that feature the 3 domains


In [ ]:
T = data.T()
D = {}
for protein in T:
    if len(T[protein]) == 3:
           D[protein] = T[protein]
print('Number of protein entries featuring FAD,NAD and Ferric transferase domains', len(D))

## Merge TMHMM & HMMR data

  * Proteins with the 3 domain types
  * Their TMHMM status


In [ ]:
merged = {}
for _id in D:
    if _id not in dataTMHMM:
        print('Missing protein ID' + _id)
    if not dataTMHMM[_id]['Htest']['status']:
        continue
    merged[_id] = {
        'hmmr' : D[_id],
        'tmhmm' : dataTMHMM[_id]
    }
    
print('Number of protein entries featuring FAD,NAD and Ferric transferase domains', len(D))
print('Number of protein featuring 2 to 7 TMH and 2 bi-histine', len(dataTMHMM))
print('Size of their intersection', len(merged))

In [ ]:
merged

In [ ]:
## Split file and do pairwise ali w/ EMBOSS
import re
saveDir="/Volumes/arwen/mobi/group/NOX_GL/seedSet/NOX_noEukaryota"
def mFastaSplitDump(data, saveDir, fileTag='default' ,distinct=True):
    c = 1
    f = None
    if not distinct:
        f = open(saveDir + '/'+ fileTag + '_all.fasta', 'w')
        
    for _id in data:
        if distinct:
            f = open(saveDir + '/'+ fileTag + '_' + str(c) + '.fasta', 'w')
        c += 1
        f.write(data[_id]['tmhmm']['fasta']['header'])
        f.write(re.sub("(.{81})", "\\1\n", data[_id]['tmhmm']['fasta']['sequence'], 0, re.DOTALL))
        if distinct:
            f.close()
    if not distinct:    
        f.close()

d = {}
for k in merged_clone:
    if not 'isNoEukaryota' in merged_clone[k]:
        continue
    if merged_clone[k]['isNoEukaryota']:
        d[k] = merged_clone[k]
        
mFastaSplitDump(d, saveDir, 'NOX_noEukaryota')

#### Inspect NCBI Taxonomy

In [ ]:
import pyproteinsExt.ontology
taxonTree = pyproteinsExt.ontology.Ontology(file='/Users/guillaumelaunay/work/databases/ontology/ncbitaxon.owl')

#### Extract TaxonID

In [ ]:

def getTaxID(datum):
    reTaxID = re.compile('OX=([\d]+)')
    m = reTaxID.search(datum['tmhmm']['fasta']['header'])
    if not m:
        raise ValueError('Cant parse taxid from', datum['tmhmm']['fasta']['header'])
    datum['taxid'] = m.groups()[0]
    
for _id in merged:
    getTaxID(merged[_id])

###### Flag Non Eukaryota phylum members

In [ ]:
cnt = 0
u = 0
for _id in merged:
    u += 1
    taxid=merged[_id]['taxid']
    n = taxonTree.onto.search(iri='http://purl.obolibrary.org/obo/NCBITaxon_' + taxid)
    if not n:
        print ('Cant find Taxon node for', taxid)
        continue

    bool=True
    for t in taxonTree._getLineage(n[0]):
        if not t.label:
            continue
        if t.label[0] == 'Eukaryota':
            bool=False
            break
    if bool:
        cnt += 1
    merged[_id]['isNoEukaryota'] = bool      

print("Total number of bacterial sequences", cnt, u)


In [ ]:
#### Cull for prokaryotic proteins (original 996)

#### Use proteins as seeds for blast ()

#### --> Tree reconstruction

#### Additional PFAM annotation

#### Sequence clustering

#### Profile génétique



##### De/Serialize the data structure


In [ ]:
import pickle, time
import time

def save(data, tag=None):
    saveDir="/Users/guillaumelaunay/work/projects/NOX"
    timestr = time.strftime("%Y%m%d-%H%M%S")
    fTag = "NOX_annotation_" + tag + "_" if tag else "NOX_annotation_"
    fSerialDump = fTag + timestr + ".pickle"
    with open(saveDir + '/' + fSerialDump, 'wb') as f:
        pickle.dump(data, f)
    print('data structure saved to', saveDir + '/' + fSerialDump)

def load(fileName):
    saveDir="/Users/guillaumelaunay/work/projects/NOX"
    d = pickle.load( open(saveDir + "/" + fileName, "rb" ) )
    print("restore a annotated container of ", len(d), "elements")
    return d

### Read in additional PFAM annotations // Erase previous
  1. restore annotated data structure
  1. import a complete PFAM scan of "isNoEukaryota" entries
  2. replace the 'hmmr' field w/ this one
  3. pickle it

In [ ]:
merged_restore = load('NOX_annotation_20180619-165729.pickle')

In [ ]:
merged_restore['tr|A0A136KU56|A0A136KU56_9CHLR']

In [ ]:
%autoreload 2
import pyproteinsExt.hmmrContainerFactory as hm
fileName="/Volumes/arwen/mobi/group/NOX_GL/toEMBOSS_hmmscan.out"
#fileName="/tmp/hmmscan.out"
hscan = hm.parse(inputFile=fileName)
print( len(hscan.T()), 'proteins to reannotate' )
for e in hscan.T():
    merged_restore[e]['hmmr'] = hscan.T()[e]

In [ ]:
save(merged_restore, tag='fullPFAM')

##### Comparing to regExp detection

In [ ]:
import re
reMotifNADPH = re.compile('G[ISVL]G[VIAF][TAS][PYTA]')
reMotifFAD = re.compile('H[PSA]F[TS][LIMV]')

NAD_miss = 0
FAD_miss = 0
Both_miss = 0
for p in merged:
    seq = merged[p]['tmhmm']['fasta']['sequence']
    m = reMotifNADPH.search(seq)
    n = reMotifFAD.search(seq)
    merged[p]['NADPH_reg'] = True if m else False
    merged[p]['FAD_reg']   = True if n else False

    if not m:
        NAD_miss += 1
        if not n:
            Both_miss += 1
    if not n:
        FAD_miss += 1

print('Total Number of filtered sequence', len(merged))
print('Number of negative to:')
print('*The NAD pattern',str(NAD_miss), '\n*The FAD pattern', str(FAD_miss), '\n*Both patterns ', Both_miss)